In [10]:
from cvxpy import *
import cvxpy
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from yahoo_finance import Share
np.set_printoptions(suppress=True)


# installed_solvers() => ['ECOS_BB', 'SCS', 'ECOS', 'LS']
# w.T*C*w =  quad_form(w, C)

"""
the sample covariance matrix – the one we derive from the historical returns – is bound to have a lot of estimation error and therefore a lower performance in the out-of-sample when compared with naively optimized portfolios (for example equally weight portfolios), especially when there are a high number of uncorrelated strategies. 
 I want to minimize Risk and I want the sum of weights to be 1 and all of them to be greater than 0.
"""
# ADD: limit to inidividual position size
# ADD: % of cash in a portfolio
# ADD: Market-neutral (sum of betas = 0)

#def calculate_portfolio(cvxtype, returns_function, long_only, exp_return = 0.15, 
#                        selected_solver='SCS', max_pos_size = 0.15):
    
def calculate_portfolio(cvxtype, returns_function, long_only, exp_return, 
                        selected_solver, max_pos_size):
    assert cvxtype in ['minimize_risk','maximize_return']
    # Variables:
    # mu is the vector of expected returns.
    # sigma is the covariance matrix.
    # gamma is a Parameter that trades off risk and return.
    # x is a vector of stock holdings as fractions of total assets.
    gamma = Parameter(sign="positive")
    gamma.value = 1
    returns, stocks, betas = returns_function
    print ("returns {}".format(str(returns.shape)))
    #print "betas {}".format(betas)
        
    cov_mat = returns.cov()
    Sigma = cov_mat.values # np.asarray(cov_mat.values) 
    w = Variable(len(cov_mat))  # #number of stocks for portfolio weights
    print ("w: " + str(w))
    risk = quad_form(w, Sigma)  #expected_variance => w.T*C*w =  quad_form(w, C)
    num_stocks = len(cov_mat)
    
    if cvxtype == 'minimize_risk': # Minimize portfolio risk / portfolio variance
        if long_only == True:
            prob = Problem(Minimize(risk), [sum_entries(w) == 1, w > 0 ])  # Long only
        else:
            prob = Problem(Minimize(risk), [sum_entries(w) == 1]) # Long / short 
    
    elif cvxtype == 'maximize_return': # Maximize portfolio return given required level of risk
        #mu = np.array([0.15,0.15,0.15,0.15,0.15,0.15,0.15]) #Expected return for each stock
        #expected_return = mu*x
        #risk = quad_form(x, sigma)
        #objective = Maximize(expected_return - gamma*risk)
        #p = Problem(objective, [sum_entries(x) == 1])
        #result = p.solve()

        #######
        mu = np.array([exp_return]*len(cov_mat)) # mu is the vector of expected returns.
        expected_return = np.reshape(mu,(-1,1)).T * w  # w is a vector of stock holdings as fractions of total assets.   
        objective = Maximize(expected_return - gamma*risk) # Maximize(expected_return - expected_variance)
        if long_only == True:
            constraints = [sum_entries(w) == 1, w > 0]
        else:
            #constraints = [sum_entries(w) == 1]    
            constraints = []
            #constraints.append(cvxpy.abs(w) <= 0.2)
            #constraints=[sum_entries(w) == 1]
            #for i in range(len(cov_mat)):
            #    constraints.extend([ w[i] < max_pos_size, w[i] > -max_pos_size])
            #for i in range(len(cov_mat)):
            #    constraints.extend([ cvxpy.abs(w[i]) <= max_pos_size])
            #constraints=[sum(w) == 1, w < max_pos_size ]
            constraints=[sum_entries(w) == 1,w <= max_pos_size, w >= -max_pos_size]
        prob = Problem(objective, constraints)

    prob.solve(solver=selected_solver)
    
    weights = []
    for weight in w.value:
        weights.append(float(weight[0]))
        
    if cvxtype == 'maximize_return':
        print("# The optimal expected return.")
        print(expected_return.value)

    print("# The optimal risk.")
    print(risk.value*100, " %")
    return weights


def getReturns(stocks = 'MSFT,AAPL,NFLX,JPM,UVXY,RSX,TBT', period_days = 100, end = '2016-12-09' ):
    stocks = stocks.split(",")
    index = 'SPY'
    stocks.append(index)
    if end is None:
        end = datetime.today().strftime("%Y-%m-%d")
    start = (datetime.today() - timedelta(period_days)).strftime("%Y-%m-%d")
        
    i = 0; w= pd.DataFrame();t = []
    for s in stocks:
        z = Share(s)
        px = pd.DataFrame(z.get_historical(start,end))[['Close','Date']]
        px['Close']=px['Close'].astype(float)
        px.index = px['Date']
        del px['Date']
        px.columns = [s]
        t.append(px)
    w = pd.concat(t,axis=1, join='inner')
    w = w.sort_index().pct_change()  #returns => w.cov() covariance matrix of returns
    #calculate betas
    betas = []
    for s in stocks:
        if s != index:
            col = np.column_stack((w[s],w[index]))
            b = np.cov(col)/np.var(w[index])
            betas.append(b)  
    stocks.remove(index)
    del w[index]
    returns = w
    print (stocks)
    return returns,stocks,np.round(betas,4)

#betas = [np.random.uniform(-1,1) for _ in range(10)]
    

In [11]:
ret = getReturns('SHLD,AAPL,NFLX,RSX,TBT,EEM,UVXY,GME,AMZN,SVXY',300,"2017-02-02")

p = calculate_portfolio(cvxtype='maximize_return',
                        returns_function=ret,
                        long_only=False,
                        exp_return=0.20,
                        selected_solver='SCS',
                        max_pos_size=0.50)
print(np.round(p,2))


['SHLD', 'AAPL', 'NFLX', 'RSX', 'TBT', 'EEM', 'UVXY', 'GME', 'AMZN', 'SVXY']
returns (207, 10)
w: var46
# The optimal expected return.
0.20046494502594359
# The optimal risk.
0.0145986955482  %
[ 0.1   0.12  0.11  0.12  0.12  0.12  0.01  0.11  0.12  0.09]


In [12]:
ret[0][-10:]

,SHLD,AAPL,NFLX,RSX,TBT,EEM,UVXY,GME,AMZN,SVXY
Date,,,,,,,,,,
2017-01-20,0.043624,0.001837,0.001373,0.003810,0.003744,0.001926,-0.078221,0.011583,-0.000878,0.039150
2017-01-23,0.000000,0.000667,-0.008730,0.004269,-0.020393,0.014827,-0.015845,0.014843,0.011814,0.008112
2017-01-24,0.010718,-0.000916,0.019798,0.012282,0.013709,0.006223,-0.093381,-0.002507,0.005575,0.046401
2017-01-25,-0.069989,0.015921,-0.004211,0.004200,0.025294,0.011024,-0.048934,0.012568,0.017120,0.024778
2017-01-26,-0.092360,0.000492,-0.004014,0.000929,-0.006106,-0.004255,0.012863,0.002069,0.003144,-0.005169
2017-01-27,-0.067839,0.000082,0.025115,0.025070,-0.007373,0.000801,-0.023761,0.003716,-0.004028,0.011901
2017-01-30,-0.075472,-0.002624,-0.008635,-0.017663,0.005942,-0.005338,0.057491,-0.005348,-0.006449,-0.028408
2017-01-31,0.017493,-0.002302,-0.003611,-0.014292,-0.012798,0.001878,-0.011111,0.012821,-0.008309,0.004603
2017-02-01,-0.047278,0.060981,0.000497,0.010290,0.011468,0.000536,-0.031300,-0.007758,0.010771,0.017480
